# 使用 llmcompressor 对 Qwen2.5-7B-Instruct 做 GPTQ & AWQ 量化

本 Notebook 演示如何基于 `llmcompressor` 库，对 `Qwen/Qwen2.5-7B-Instruct` 分别应用 **GPTQ** 与 **AWQ** 量化，
并加载量化后的检查点进行对话推理。代码风格沿用你当前的 Notebook 写法。

这里直接使用 llmcompressor 提供的统一 API，而不是手动调用 `auto-gptq` / `awq` 等底层库。

In [1]:
# 安装依赖（如环境已具备可跳过）
%pip install -q "transformers>=4.54.0,<=4.57.3" accelerate llmcompressor datasets

# llmcompressor 内部已经集成 GPTQ / AWQ 等算法，一般无需手动安装 auto-gptq / awq 等库。

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from llmcompressor.modifiers.quantization import GPTQModifier
from llmcompressor import oneshot

# 基础配置：Qwen2.5-7B 指令模型
base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"

# 设备配置（优先使用 GPU）
device = "cuda" if torch.cuda.is_available() else "cpu"
device

## 一、使用 llmcompressor + GPTQ 对 Qwen2.5-7B-Instruct 进行量化

这一节使用 `GPTQModifier` + `oneshot` API，对基座模型做权重量化（示例采用 `W4A16` 配置，仅作教学演示）。

默认使用公开数据集 `open_platypus` 作为校准集，你也可以在熟悉流程后替换为自己的中文数据集。

In [ ]:
# 使用 llmcompressor 对 Qwen2.5-7B-Instruct 做 GPTQ 量化

# 量化后模型输出目录（可根据需要调整路径）
gptq_out_dir = "models/qwen2.5-1.5b-instruct-gptq-llmc"

gptq_recipe = [
    GPTQModifier(
        scheme="W4A16",      # 权重 4bit，激活保持 16bit
        targets="Linear",    # 只量化线性层
        ignore=["lm_head"],  # 通常不量化输出头
    ),
]

oneshot(
    model=base_model_id,
    dataset="open_platypus",      # 内置公开数据集，方便快速演示
    recipe=gptq_recipe,
    output_dir=gptq_out_dir,
    max_seq_length=2048,
    num_calibration_samples=128,   # 为了速度，这里只取少量校准样本
)

gptq_out_dir

`torch_dtype` is deprecated! Use `dtype` instead!


Tokenizing:   0%|          | 0/24926 [00:00<?, ? examples/s]

2025-12-19T03:57:26.694798+0800 | reset | INFO - Compression lifecycle reset
2025-12-19T03:57:26.702409+0800 | from_modifiers | INFO - Creating recipe from modifiers
2025-12-19T03:57:26.741739+0800 | initialize | INFO - Compression lifecycle initialized for 1 modifiers
2025-12-19T03:57:26.741739+0800 | IndependentPipeline | INFO - Inferred `SequentialPipeline` for `GPTQModifier`


(1/29): Calibrating: 100%|██████████| 128/128 [00:07<00:00, 16.95it/s]

2025-12-19T03:57:34.951606+0800 | compress_modules | INFO - Quantizing model.layers.0.self_attn.q_proj using 128 samples


...

(29/29): Propagating: 100%|██████████| 128/128 [00:00<00:00, 854.91it/s]

2025-12-19T04:06:34.668162+0800 | finalize | INFO - Compression lifecycle finalized for 1 modifiers


2025-12-19T04:06:34.722376+0800 | get_model_compressor | INFO - skip_sparsity_compression_stats set to True. Skipping sparsity compression statistic calculations. No sparsity compressor will be applied.


Compressing model: 196it [00:03, 52.05it/s]


'models/qwen2.5-1.5b-instruct-gptq-llmc'

In [ ]:
# 加载 GPTQ 量化后的检查点做推理

gptq_tokenizer = AutoTokenizer.from_pretrained(gptq_out_dir, trust_remote_code=True)
if gptq_tokenizer.pad_token_id is None:
    gptq_tokenizer.pad_token = gptq_tokenizer.eos_token

gptq_model = AutoModelForCausalLM.from_pretrained(
    gptq_out_dir,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
)
gptq_model.eval()

gptq_tokenizer.pad_token, gptq_tokenizer.eos_token, gptq_tokenizer.pad_token_id, gptq_tokenizer.eos_token_id

The tokenizer you are loading from 'models/qwen2.5-1.5b-instruct-gptq-llmc' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Compressing model: 196it [00:00, 1131.44it/s]


('<|endoftext|>', '<|im_end|>', 151643, 151645)

In [5]:
# 检查 GPTQ 量化是否生效：查看第 0 层 self_attn.q_proj 的类型和配置
layer = gptq_model.model.layers[0].self_attn.q_proj
print("GPTQ q_proj layer type:", type(layer))
print("GPTQ quantization_config:", getattr(gptq_model.config, "quantization_config", None))

GPTQ q_proj layer type: <class 'compressed_tensors.linear.compressed_linear.CompressedLinear'>
GPTQ quantization_config: CompressedTensorsConfig {
  "config_groups": {
    "group_0": {
      "format": "pack-quantized",
      "input_activations": null,
      "output_activations": null,
      "targets": [
        "Linear"
      ],
      "weights": {
        "actorder": "static",
        "block_structure": null,
        "dynamic": false,
        "group_size": 128,
        "num_bits": 4,
        "observer": "minmax",
        "observer_kwargs": {},
        "scale_dtype": null,
        "strategy": "group",
        "symmetric": true,
        "type": "int",
        "zp_dtype": null
      }
    }
  },
  "format": "pack-quantized",
  "global_compression_ratio": null,
  "ignore": [
    "lm_head"
  ],
  "kv_cache_scheme": null,
  "quantization_status": "compressed"
}



# GPTQ 量化模型对话推理示例

In [5]:
GPTQ_TEST_QUERIES = [
    "用两三句话解释一下什么是量子计算？",
    "Give me a brief introduction to large language models in English.",
]

@torch.no_grad()
def gptq_chat(question: str) -> str:
    msgs = [
        {"role": "system", "content": "你是一名 AI 助手，回答准确、简洁。"},
        {"role": "user", "content": question},
    ]
    input_ids = gptq_tokenizer.apply_chat_template(
        msgs,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(gptq_model.device)

    gen_ids = gptq_model.generate(
        input_ids=input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        eos_token_id=gptq_tokenizer.eos_token_id,
        pad_token_id=gptq_tokenizer.pad_token_id,
    )
    out_ids = gen_ids[0, input_ids.shape[-1]:]
    return gptq_tokenizer.decode(out_ids, skip_special_tokens=True).strip()

for q in GPTQ_TEST_QUERIES:
    ans = gptq_chat(q)
    print(f"[GPTQ] Q: {q}\nA: {ans}\n" + "-" * 60)

[GPTQ] Q: 用两三句话解释一下什么是量子计算？
A: 量子计算是利用量子位（qubits）代替经典比特进行信息处理的一种计算方式。它利用量子叠加、纠缠等特性来实现超越传统计算机的并行处理能力，可以高效地解决某些特定问题。通过使用量子比特，量子计算机可以在短时间内完成复杂的数学运算和模拟物理系统，对于密码破解、优化算法等领域具有巨大潜力。尽管目前的技术还处于发展阶段，但未来有望成为一种强大的计算工具。
------------------------------------------------------------
[GPTQ] Q: Give me a brief introduction to large language models in English.
A: Large Language Models (LLMs) are advanced artificial intelligence systems that can understand and generate human-like text across various domains, including natural language processing tasks such as translation, summarization, question-answering, and more.

These models use deep learning algorithms, which enable them to learn from vast amounts of data and improve their performance over time. The most prominent example is the GPT (Generative Pretrained Transformer), developed by OpenAI, which has been used for many applications like language generation, image description, and even creative writing.

One key feature of LLMs is their ability to hand

In [6]:
# 释放 GPTQ 量化模型占用的显存
del gptq_model
torch.cuda.empty_cache()

## 二、使用 llmcompressor + AWQ 对 Qwen2.5-7B-Instruct 进行量化

In [ ]:
from llmcompressor.modifiers.awq import AWQModifier
# 使用 llmcompressor 对 Qwen2.5-7B-Instruct 做 AWQ 量化

awq_out_dir = "models/qwen2.5-1.5b-instruct-awq-llmc"

awq_recipe = [
    AWQModifier(
        scheme="W4A16",
        targets="Linear",
        ignore=["lm_head"],
    ),
]

oneshot(
    model=base_model_id,
    dataset="open_platypus",
    recipe=awq_recipe,
    output_dir=awq_out_dir,
    max_seq_length=2048,
    num_calibration_samples=128,
)

awq_out_dir

Tokenizing:   0%|          | 0/24926 [00:00<?, ? examples/s]

2025-12-19T04:08:32.087210+0800 | reset | INFO - Compression lifecycle reset
2025-12-19T04:08:32.099173+0800 | from_modifiers | INFO - Creating recipe from modifiers
2025-12-19T04:08:32.139239+0800 | on_initialize | INFO - No AWQModifier.mappings provided, inferring from model...
2025-12-19T04:08:32.146092+0800 | _set_resolved_mappings | WARNING - skipping AWQ for model.layers.0.self_attn.v_proj for mapping AWQMapping(smooth_layer='re:.*v_proj$', balance_layers=['re:.*o_proj$']) because found incompatible balance layers
2025-12-19T04:08:32.147094+0800 | _set_resolved_mappings | WARNING - skipping AWQ for model.layers.1.self_attn.v_proj for mapping AWQMapping(smooth_layer='re:.*v_proj$', balance_layers=['re:.*o_proj$']) because found incompatible balance layers
2025-12-19T04:08:32.148103+0800 | _set_resolved_mappings | WARNING - skipping AWQ for model.layers.2.self_attn.v_proj for mapping AWQMapping(smooth_layer='re:.*v_proj$', balance_layers=['re:.*o_proj$']) because found incompatible

(29/29): Calibrating: 100%|██████████| 128/128 [00:00<00:00, 306.53it/s]
Smoothing: 0it [00:00, ?it/s]
(29/29): Propagating: 100%|██████████| 128/128 [00:00<00:00, 904.36it/s]
Smoothing: 0it [00:00, ?it/s]
Calibrating weights: 196it [00:08, 22.98it/s]

2025-12-19T09:17:58.322208+0800 | finalize | INFO - Compression lifecycle finalized for 1 modifiers


2025-12-19T09:17:58.390265+0800 | get_model_compressor | INFO - skip_sparsity_compression_stats set to True. Skipping sparsity compression statistic calculations. No sparsity compressor will be applied.


Compressing model: 196it [00:03, 53.78it/s]


'models/qwen2.5-1.5b-instruct-awq-llmc'

In [ ]:
# 加载 AWQ 量化后的检查点并做对话推理

awq_tokenizer = AutoTokenizer.from_pretrained(awq_out_dir, trust_remote_code=True)
if awq_tokenizer.pad_token_id is None:
    awq_tokenizer.pad_token = awq_tokenizer.eos_token

awq_model = AutoModelForCausalLM.from_pretrained(
    awq_out_dir,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
)
awq_model.eval()

The tokenizer you are loading from 'models/qwen2.5-1.5b-instruct-awq-llmc' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Compressing model: 196it [00:00, 1146.78it/s]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): CompressedLinear(in_features=1536, out_features=1536, bias=True)
          (k_proj): CompressedLinear(in_features=1536, out_features=256, bias=True)
          (v_proj): CompressedLinear(in_features=1536, out_features=256, bias=True)
          (o_proj): CompressedLinear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): CompressedLinear(in_features=1536, out_features=8960, bias=False)
          (up_proj): CompressedLinear(in_features=1536, out_features=8960, bias=False)
          (down_proj): CompressedLinear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)

In [7]:
# 检查 AWQ 量化是否生效：查看第 0 层 self_attn.q_proj 的类型和配置
layer = awq_model.model.layers[0].self_attn.q_proj
print("AWQ q_proj layer type:", type(layer))
print("AWQ quantization_config:", getattr(awq_model.config, "quantization_config", None))

AWQ q_proj layer type: <class 'compressed_tensors.linear.compressed_linear.CompressedLinear'>
AWQ quantization_config: CompressedTensorsConfig {
  "config_groups": {
    "group_0": {
      "format": "pack-quantized",
      "input_activations": null,
      "output_activations": null,
      "targets": [
        "Linear"
      ],
      "weights": {
        "actorder": null,
        "block_structure": null,
        "dynamic": false,
        "group_size": 128,
        "num_bits": 4,
        "observer": "minmax",
        "observer_kwargs": {},
        "scale_dtype": null,
        "strategy": "group",
        "symmetric": true,
        "type": "int",
        "zp_dtype": null
      }
    }
  },
  "format": "pack-quantized",
  "global_compression_ratio": null,
  "ignore": [
    "lm_head"
  ],
  "kv_cache_scheme": null,
  "quantization_status": "compressed"
}



In [ ]:
AWQ_TEST_QUERIES = [
    "简单说说大模型量化有什么好处？",
    "Explain in English why activation-aware weight quantization (AWQ) can help LLMs.",
]

@torch.no_grad()
def awq_chat(question: str) -> str:
    msgs = [
        {"role": "system", "content": "你是一名 AI 助手，回答准确、简洁。"},
        {"role": "user", "content": question},
    ]
    input_ids = awq_tokenizer.apply_chat_template(
        msgs,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(awq_model.device)

    gen_ids = awq_model.generate(
        input_ids=input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        eos_token_id=awq_tokenizer.eos_token_id,
        pad_token_id=awq_tokenizer.pad_token_id,
    )
    out_ids = gen_ids[0, input_ids.shape[-1]:]
    return awq_tokenizer.decode(out_ids, skip_special_tokens=True).strip()

for q in AWQ_TEST_QUERIES:
    ans = awq_chat(q)
    print(f"[AWQ] Q: {q}\nA: {ans}\n" + "-" * 60)


The tokenizer you are loading from 'models/qwen2.5-1.5b-instruct-awq-llmc' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Compressing model: 196it [00:00, 1314.02it/s]


[AWQ] Q: 简单说说大模型量化有什么好处？
A: 1. 提高计算效率：通过去除多余的参数和操作来降低模型的复杂度和内存占用。
2. 降低成本：减少计算量可以降低硬件需求和功耗。
3. 改善泛化能力：优化后的模型通常在一些任务上表现更优，能够更好地泛化到其他数据集。
4. 实现可解释性：对量化后的模型进行分析可以理解其工作原理。
5. 推广应用：适合于各种低资源环境下的部署。
------------------------------------------------------------
[AWQ] Q: Explain in English why activation-aware weight quantization (AWQ) can help LLMs.
A: Activation-aware weight quantization (AWQ) helps large language models, or Large Language Models (LLMs), by improving their performance and efficiency while reducing the amount of computation required to run them.

Here’s how AWQ works and why it is beneficial:

1. **Reducing the Size of Model Parameters**: Activation-aware weight quantization involves mapping activations from a continuous range into a smaller discrete set. This process reduces the number of parameters needed for the model, which leads to smaller and more efficient models with fewer parameters.

2. **Improving Performance**: Smaller models are often faster and use less memory, ma